### Find Affine matrix


In this notebook we try different methods to find the affine matrix that was used to transform the heart of all patients


### Methods

We used different methods to find the affine matrix namely:

1) Find the affine matrix using the elastix extension in 3d slicer, with the preset default rigid settings
2) Find the affine matrix using the elastix extension in 3d slicer, with a custom affine settings
3) Find the affine matrix by first finding the translation matrix then trying to find the rotation matrix with svd
4) Find the affine matrix by first finding the translation matrix, compare with the GT2 and trying different small rotations


### Finding the Transformation with Elastix

To determine the transformation between two image segmentations (the correctly labeled GT and the wrongly transformed GT), I used the **Elastix** tool in **3D Slicer**. Elastix is a versatile registration algorithm that computes the spatial transformation required to align one image (moving image) to another (fixed image).

For my task, I performed a **rigid/affine registration** between the correct ground truth (GT2) as the fixed image and the incorrectly transformed ground truth (GT) as the moving image.

1. **Input Images**: 
   - The fixed image was the correct segmentation.
   - The moving image was the incorrectly transformed segmentation.
   
2. **Elastix Setup**: 
   - I used a **rigid/affine transform** preset, which allows for rotation, translation, and scaling to align the images.
   - I also used a **custom affine transform** preset
   - The registration process computes a transformation matrix that maps the coordinates of the moving image to the fixed image.

3. **Output**: 
   - After running the registration, Elastix generated a transformation matrix that aligns the wrong GT with the correct GT.
   - Additionally, I enabled the option to output the **inverse transformation**, which allows us to correct the wrongly transformed segmentation by reversing the applied transformation.

This transformation was saved as a file, which I then used to correct the wrongly transformed segmentation in subsequent analysis steps.


In [1]:
import nibabel as nib
import numpy as np
from scipy.ndimage import binary_erosion

In [2]:
def calculate_dice(mask1, mask2):
    """
    Calculate Dice coefficient between two binary masks.
    """
    intersection = np.logical_and(mask1, mask2).sum()
    dice = (2. * intersection) / (mask1.sum() + mask2.sum())
    return dice

In [4]:
import os
print(os.getcwd())

/home/ragnar_vd/AI4MI/data


In [6]:
# Dice coefficient for GT2 with the transformed heart found with elastix using the preset:
# generic all

translated_GT_file = 'segthor_original/train/Patient_27/GT_corrected_rigid.nii.gz'
gt2_file = 'segthor_original/train/Patient_27/GT2.nii.gz'

translated_GT = nib.load(translated_GT_file).get_fdata()
gt2 = nib.load(gt2_file).get_fdata()

# Binarize the masks for the heart segmentation (label 2)
transformed = translated_GT
gt2_mask = (gt2 == 2)

dice = calculate_dice(transformed, gt2_mask)
print(f"Dice coefficient: {dice}")




Dice coefficient: 0.27628742049729255


In [8]:
# Dice coefficient for GT2 with the transformed heart found with elastix using the preset:
# generic rigid all
translated_GT_file = 'segthor_original/train/Patient_27/GT_transformed_2.nii.gz'
gt2_file = 'segthor_original/train/Patient_27/GT2.nii.gz'

translated_GT = nib.load(translated_GT_file).get_fdata()
gt2 = nib.load(gt2_file).get_fdata()

# Binarize the masks for the heart segmentation (label 2)
transformed = translated_GT
gt2_mask = (gt2 == 2)

dice = calculate_dice(transformed, gt2_mask)
print(f"Dice coefficient: {dice}")




Dice coefficient: 0.3499658785275581


As you can see this does not give the wanted results

So here we tried to first compute the translation matrix and then try to find the rotation matrix

In [12]:
import nibabel as nib
import numpy as np
from scipy.ndimage import affine_transform

# Define the Dice coefficient function (if not defined earlier)
def calculate_dice(mask1, mask2):
    """
    Calculate Dice coefficient between two binary masks.
    """
    intersection = np.logical_and(mask1, mask2).sum()
    dice = (2. * intersection) / (mask1.sum() + mask2.sum())
    return dice

# Load the images
wrong_GT = nib.load('segthor_original/train/Patient_27/GT_old.nii.gz')
good_GT = nib.load('segthor_original/train/Patient_27/GT2.nii.gz')

wrong_GT_data = wrong_GT.get_fdata()
good_GT_data = good_GT.get_fdata()

# Extract only the heart segmentation (label 2)
wrong_mask = (wrong_GT_data == 2)
good_mask = (good_GT_data == 2)

# Get the coordinates of the heart voxels
coords_wrong = np.array(np.nonzero(wrong_mask)).T
coords_good = np.array(np.nonzero(good_mask)).T

# Ensure both arrays have the same number of points
min_points = min(len(coords_wrong), len(coords_good))
if len(coords_wrong) > min_points:
    coords_wrong = coords_wrong[np.random.choice(len(coords_wrong), min_points, replace=False)]
else:
    coords_good = coords_good[np.random.choice(len(coords_good), min_points, replace=False)]

# Compute the centroids (center of mass)
centroid_wrong = coords_wrong.mean(axis=0)
centroid_good = coords_good.mean(axis=0)

# Compute the translation vector and reverse it
translation_vector = centroid_good - centroid_wrong
print(f"Original Computed Translation Vector: {translation_vector}")

# Store the translation vector in a 4x4 affine transformation matrix (identity matrix with translation)
affine_matrix = np.eye(4)
affine_matrix[:3, 3] = translation_vector
print(f"Affine matrix (with translation only):\n{affine_matrix}")

# Compute the inverse of the affine matrix
INV = np.linalg.inv(affine_matrix)

# Apply the inverse transformation to the wrong GT mask
transformed_wrong_mask = affine_transform(wrong_mask.astype(np.float32), INV[:3, :3], offset=INV[:3, 3])
transformed_wrong_mask = np.round(transformed_wrong_mask).astype(np.uint8)  # Ensure it's binary

# Save the transformed mask
output_file = 'segthor_original/train/Patient_27/GT_transformed_translation.nii.gz'
transformed_nii = nib.Nifti1Image(transformed_wrong_mask, wrong_GT.affine)
nib.save(transformed_nii, output_file)
print(f"Transformed image saved as {output_file}")

# Calculate Dice coefficient between the transformed wrong GT and the correct GT2
dice_coefficient = calculate_dice(transformed_wrong_mask, good_mask)
print(f"Dice Coefficient between transformed GT_old and GT2: {dice_coefficient:.4f}")


Original Computed Translation Vector: [66.58251408 38.56859134 14.99967275]
Affine matrix (with translation only):
[[ 1.          0.          0.         66.58251408]
 [ 0.          1.          0.         38.56859134]
 [ 0.          0.          1.         14.99967275]
 [ 0.          0.          0.          1.        ]]
Transformed image saved as segthor_original/train/Patient_27/GT_transformed_translation.nii.gz
Dice Coefficient between transformed GT_old and GT2: 0.9046


This already gives much better results now we only need to find the rotation matrix and at it to the affine matrix. We tried different strategies for this:
1) First we checked this in 3d slicer and tried to find the rotation here

In [13]:
# Dice coefficient for GT2 with the transformed heart found with elastix using the preset:
# generic rigid all
translated_GT_file = 'segthor_original/train/Patient_27/gt_transformed_translation.nii.gz'
gt2_file = 'segthor_original/train/Patient_27/GT2.nii.gz'

translated_GT = nib.load(translated_GT_file).get_fdata()
gt2 = nib.load(gt2_file).get_fdata()

# Binarize the masks for the heart segmentation (label 2)
transformed = translated_GT
gt2_mask = (gt2 == 2)

dice = calculate_dice(transformed, gt2_mask)
print(f"Dice coefficient: {dice}")




Dice coefficient: 0.7145870190181336


This gave worse results than expected but. So after some visually inspection we thought that there would only be a small rotation.

In [18]:
import nibabel as nib
import numpy as np
from scipy.ndimage import affine_transform

# Load the images
wrong_GT = nib.load('segthor_original/train/Patient_27/GT_old.nii.gz')
good_GT = nib.load('segthor_original/train/Patient_27/GT2.nii.gz')

wrong_GT_data = wrong_GT.get_fdata()
good_GT_data = good_GT.get_fdata()

# Extract only the heart segmentation (label 2)
wrong_mask = (wrong_GT_data == 2)
good_mask = (good_GT_data == 2)

# Get the coordinates of the heart voxels
coords_wrong = np.array(np.nonzero(wrong_mask)).T
coords_good = np.array(np.nonzero(good_mask)).T

# Ensure both arrays have the same number of points
min_points = min(len(coords_wrong), len(coords_good))
if len(coords_wrong) > min_points:
    coords_wrong = coords_wrong[np.random.choice(len(coords_wrong), min_points, replace=False)]
else:
    coords_good = coords_good[np.random.choice(len(coords_good), min_points, replace=False)]

# Compute the centroids (center of mass)
centroid_wrong = coords_wrong.mean(axis=0)
centroid_good = coords_good.mean(axis=0)

# Compute the translation vector and reverse it
translation_vector = centroid_good - centroid_wrong
print(f"Original Computed Translation Vector: {translation_vector}")

# Store the translation vector in a 4x4 affine transformation matrix (identity matrix with translation)
affine_matrix = np.eye(4)
affine_matrix[:3, 3] = translation_vector
print(f"Affine matrix (with translation only):\n{affine_matrix}")

TR = affine_matrix

# Calculate the Dice coefficient
def calculate_dice(mask1, mask2):
    """
    Calculate Dice coefficient between two binary masks.
    """
    intersection = np.logical_and(mask1, mask2).sum()
    dice = (2. * intersection) / (mask1.sum() + mask2.sum())
    return dice

# Loop over degrees from 25 to 30
for DEG in range(-10,10):
    ϕ = - DEG / 180 * np.pi  # Convert degree to radians for rotation

    # 3x3 Rotation matrix around the Z-axis
    RO = np.asarray([[np.cos(ϕ), -np.sin(ϕ), 0, 0],  
                     [np.sin(ϕ),  np.cos(ϕ), 0, 0],  
                     [     0,         0,     1, 0],  
                     [     0,         0,     0, 1]])

    # Constants for the centroids
    X_bar = 0
    Y_bar = 0
    Z_bar = 0
    C1 = np.asarray([[1, 0, 0, X_bar],
                     [0, 1, 0, Y_bar],
                     [0, 0, 1, Z_bar],
                     [0, 0, 0,    1]])
    C2 = np.linalg.inv(C1)

    # Full affine matrix (translation + rotation)
    AFF = C1 @ RO @ C2 @ TR
    INV = np.linalg.inv(AFF)

    # Apply the inverse transformation to the wrong GT mask
    transformed_wrong_mask = affine_transform(wrong_mask.astype(np.float32), INV[:3, :3], offset=INV[:3, 3])
    transformed_wrong_mask = np.round(transformed_wrong_mask).astype(np.uint8)  # Ensure it's binary

    # Calculate Dice coefficient between the transformed wrong GT and the correct GT2
    dice_coefficient = calculate_dice(transformed_wrong_mask, good_mask)
    print(f"Dice Coefficient for degree {DEG} between transformed GT_old and GT2: {dice_coefficient:.4f}")


Original Computed Translation Vector: [66.58220038 38.56809934 14.99948092]
Affine matrix (with translation only):
[[ 1.          0.          0.         66.58220038]
 [ 0.          1.          0.         38.56809934]
 [ 0.          0.          1.         14.99948092]
 [ 0.          0.          0.          1.        ]]
Dice Coefficient for degree -10 between transformed GT_old and GT2: 0.3214
Dice Coefficient for degree -9 between transformed GT_old and GT2: 0.3786
Dice Coefficient for degree -8 between transformed GT_old and GT2: 0.4386
Dice Coefficient for degree -7 between transformed GT_old and GT2: 0.5007
Dice Coefficient for degree -6 between transformed GT_old and GT2: 0.5644
Dice Coefficient for degree -5 between transformed GT_old and GT2: 0.6294
Dice Coefficient for degree -4 between transformed GT_old and GT2: 0.6955
Dice Coefficient for degree -3 between transformed GT_old and GT2: 0.7617
Dice Coefficient for degree -2 between transformed GT_old and GT2: 0.8243
Dice Coeffici

KeyboardInterrupt: 

This gave very reasonable results 

We also tried to found the rotation in python

In [17]:
import SimpleITK as sitk
import numpy as np
import nibabel as nib

def register_images_rigid(fixed_img_file, moving_img_file, output_transform_file, replace_index=None):
    """
    Register two images using a rigid transformation (translation + rotation) and save the transform.
    """
    # Read the images
    fixed_img = sitk.ReadImage(fixed_img_file, sitk.sitkFloat32)
    moving_img = sitk.ReadImage(moving_img_file, sitk.sitkFloat32)

    # If specific label replacement is needed
    if replace_index:
        fixed_img_array = sitk.GetArrayFromImage(fixed_img)
        fixed_img_array = (fixed_img_array == replace_index).astype(np.float32)
        fixed_img = sitk.GetImageFromArray(fixed_img_array)

        moving_img_array = sitk.GetArrayFromImage(moving_img)
        moving_img_array = (moving_img_array == replace_index).astype(np.float32)
        moving_img = sitk.GetImageFromArray(moving_img_array)

    # Setup the registration method with rigid transform
    registration = sitk.ImageRegistrationMethod()
    registration.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration.SetMetricSamplingStrategy(registration.RANDOM)
    registration.SetMetricSamplingPercentage(0.1)  # Increase sampling percentage
    registration.SetInterpolator(sitk.sitkNearestNeighbor)
    registration.SetOptimizerAsGradientDescent(
        learningRate=0.5,  # Reduce learning rate for finer steps
        numberOfIterations=500,  # Increase number of iterations
        convergenceMinimumValue=1e-7,
        convergenceWindowSize=10,
    )
    registration.SetOptimizerScalesFromPhysicalShift()

    # Initialize the transform as a rigid (Euler 3D) transform
    rigid_transform = sitk.CenteredTransformInitializer(
        fixed_img, moving_img, sitk.Euler3DTransform(), sitk.CenteredTransformInitializerFilter.GEOMETRY
    )
    registration.SetInitialTransform(rigid_transform)

    # Use a pyramid schedule to help with convergence
    registration.SetShrinkFactorsPerLevel(shrinkFactors=[4, 2, 1])
    registration.SetSmoothingSigmasPerLevel(smoothingSigmas=[2, 1, 0])
    registration.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # Perform the registration
    transform = registration.Execute(fixed_img, moving_img)

    # Save the transformation
    sitk.WriteTransform(transform, output_transform_file)
    print("Rigid registration complete and transform saved.")

    return transform


def apply_transform(moving_img_file, output_img_file, transform_file):
    """
    Apply the saved transformation to the moving image.
    """
    moving_img = sitk.ReadImage(moving_img_file, sitk.sitkFloat32)
    transform = sitk.ReadTransform(transform_file)

    # Resample the moving image
    resampled_img = sitk.Resample(moving_img, moving_img, transform, sitk.sitkNearestNeighbor, 0.0)

    # Save the transformed image
    sitk.WriteImage(resampled_img, output_img_file)
    print(f"Transformed image saved as {output_img_file}")


def calculate_dice(mask1, mask2):
    """
    Calculate Dice coefficient between two binary masks.
    """
    intersection = np.logical_and(mask1, mask2).sum()
    dice = (2. * intersection) / (mask1.sum() + mask2.sum())
    return dice


# Paths to the NIfTI images and the transform
fixed_img_file = 'segthor_original/train/Patient_27/GT2.nii.gz'  # GT2 is the fixed image
moving_img_file = 'segthor_original/train/Patient_27/GT_old.nii.gz'  # GT_old is the moving image
output_transform_file = 'segthor_train/train/Patient_27/rigid_transform.tfm'
output_transformed_img_file = 'segthor_train/train/Patient_27/GT_corrected_rigid.nii.gz'

# Register images using rigid transform
transform = register_images_rigid(fixed_img_file, moving_img_file, output_transform_file, replace_index=2)

# Apply the transformation to the moving image
apply_transform(moving_img_file, output_transformed_img_file, output_transform_file)

# Load the fixed and transformed images for Dice calculation
fixed_img = nib.load(fixed_img_file).get_fdata()
transformed_img = nib.load(output_transformed_img_file).get_fdata()

# Binarize the heart masks (label 2)
fixed_mask = (fixed_img == 2)
transformed_mask = (transformed_img == 2)

# Calculate Dice coefficient
dice_coefficient = calculate_dice(fixed_mask, transformed_mask)
print(f"Dice Coefficient between the rigidly transformed GT and GT2: {dice_coefficient:.4f}")



Rigid registration complete and transform saved.
Transformed image saved as segthor_train/train/Patient_27/GT_corrected_rigid.nii.gz
Dice Coefficient between the rigidly transformed GT and GT2: 0.5852


This was not a succes so we found in Elastix again the centroids of the rotation FixedParameters: **-30.68731694  21.74786194  20.2083591** and tried to find the rotation with these centroids

With the centroids found in the provided affine matrix

In [19]:
import nibabel as nib
import numpy as np
from scipy.ndimage import affine_transform

# Load the images
wrong_GT = nib.load('segthor_original/train/Patient_27/GT_old.nii.gz')
good_GT = nib.load('segthor_original/train/Patient_27/GT2.nii.gz')

wrong_GT_data = wrong_GT.get_fdata()
good_GT_data = good_GT.get_fdata()

# Extract only the heart segmentation (label 2)
wrong_mask = (wrong_GT_data == 2)
good_mask = (good_GT_data == 2)

# Get the coordinates of the heart voxels
coords_wrong = np.array(np.nonzero(wrong_mask)).T
coords_good = np.array(np.nonzero(good_mask)).T

# Ensure both arrays have the same number of points
min_points = min(len(coords_wrong), len(coords_good))
if len(coords_wrong) > min_points:
    coords_wrong = coords_wrong[np.random.choice(len(coords_wrong), min_points, replace=False)]
else:
    coords_good = coords_good[np.random.choice(len(coords_good), min_points, replace=False)]

# Compute the centroids (center of mass)
centroid_wrong = coords_wrong.mean(axis=0)
centroid_good = coords_good.mean(axis=0)

# Compute the translation vector and reverse it
translation_vector = centroid_good - centroid_wrong
print(f"Original Computed Translation Vector: {translation_vector}")

# Store the translation vector in a 4x4 affine transformation matrix (identity matrix with translation)
affine_matrix = np.eye(4)
affine_matrix[:3, 3] = translation_vector
print(f"Affine matrix (with translation only):\n{affine_matrix}")

TR = affine_matrix

# Calculate the Dice coefficient
def calculate_dice(mask1, mask2):
    """
    Calculate Dice coefficient between two binary masks.
    """
    intersection = np.logical_and(mask1, mask2).sum()
    dice = (2. * intersection) / (mask1.sum() + mask2.sum())
    return dice

# Hardcoded centroids as provided (X_bar = 268, Y_bar = 210, Z_bar = 0)
X_bar = 275
Y_bar = 200
Z_bar = 0

# Loop over degrees from -10 to 10
for DEG in range(-10, 10):
    ϕ = - DEG / 180 * np.pi  # Convert degree to radians for rotation

    # 3x3 Rotation matrix around the Z-axis
    RO = np.asarray([[np.cos(ϕ), -np.sin(ϕ), 0, 0],  
                     [np.sin(ϕ),  np.cos(ϕ), 0, 0],  
                     [     0,         0,     1, 0],  
                     [     0,         0,     0, 1]])

    # Constants for the centroids (using the hardcoded values)
    C1 = np.asarray([[1, 0, 0, X_bar],
                     [0, 1, 0, Y_bar],
                     [0, 0, 1, Z_bar],
                     [0, 0, 0,    1]])
    C2 = np.linalg.inv(C1)

    # Full affine matrix (translation + rotation)
    AFF = C1 @ RO @ C2 @ TR
    INV = np.linalg.inv(AFF)

    # Apply the inverse transformation to the wrong GT mask
    transformed_wrong_mask = affine_transform(wrong_mask.astype(np.float32), INV[:3, :3], offset=INV[:3, 3])
    transformed_wrong_mask = np.round(transformed_wrong_mask).astype(np.uint8)  # Ensure it's binary

    # Calculate Dice coefficient between the transformed wrong GT and the correct GT2
    dice_coefficient = calculate_dice(transformed_wrong_mask, good_mask)
    print(f"Dice Coefficient for degree {DEG} between transformed GT_old and GT2: {dice_coefficient:.4f}")


Original Computed Translation Vector: [66.58171063 38.56887797 14.99938387]
Affine matrix (with translation only):
[[ 1.          0.          0.         66.58171063]
 [ 0.          1.          0.         38.56887797]
 [ 0.          0.          1.         14.99938387]
 [ 0.          0.          0.          1.        ]]
Dice Coefficient for degree -10 between transformed GT_old and GT2: 0.8628
Dice Coefficient for degree -9 between transformed GT_old and GT2: 0.8670
Dice Coefficient for degree -8 between transformed GT_old and GT2: 0.8713
Dice Coefficient for degree -7 between transformed GT_old and GT2: 0.8755
Dice Coefficient for degree -6 between transformed GT_old and GT2: 0.8798
Dice Coefficient for degree -5 between transformed GT_old and GT2: 0.8840
Dice Coefficient for degree -4 between transformed GT_old and GT2: 0.8882
Dice Coefficient for degree -3 between transformed GT_old and GT2: 0.8923
Dice Coefficient for degree -2 between transformed GT_old and GT2: 0.8964
Dice Coeffici

In [22]:
import nibabel as nib
import numpy as np
from scipy.ndimage import affine_transform

# Load the images
wrong_GT = nib.load('segthor_original/train/Patient_27/GT_old.nii.gz')
good_GT = nib.load('segthor_original/train/Patient_27/GT2.nii.gz')

wrong_GT_data = wrong_GT.get_fdata()
good_GT_data = good_GT.get_fdata()

# Extract only the heart segmentation (label 2)
wrong_mask = (wrong_GT_data == 2)
good_mask = (good_GT_data == 2)

# Get the coordinates of the heart voxels
coords_wrong = np.array(np.nonzero(wrong_mask)).T
coords_good = np.array(np.nonzero(good_mask)).T

# Ensure both arrays have the same number of points
min_points = min(len(coords_wrong), len(coords_good))
if len(coords_wrong) > min_points:
    coords_wrong = coords_wrong[np.random.choice(len(coords_wrong), min_points, replace=False)]
else:
    coords_good = coords_good[np.random.choice(len(coords_good), min_points, replace=False)]

# Compute the centroids (center of mass)
centroid_wrong = coords_wrong.mean(axis=0)
centroid_good = coords_good.mean(axis=0)

# Compute the translation vector and reverse it
translation_vector = centroid_good - centroid_wrong
print(f"Original Computed Translation Vector: {translation_vector}")

# Store the translation vector in a 4x4 affine transformation matrix (identity matrix with translation)
affine_matrix = np.eye(4)
affine_matrix[:3, 3] = translation_vector
print(f"Affine matrix (with translation only):\n{affine_matrix}")

TR = affine_matrix

# Calculate the Dice coefficient
def calculate_dice(mask1, mask2):
    """
    Calculate Dice coefficient between two binary masks.
    """
    intersection = np.logical_and(mask1, mask2).sum()
    dice = (2. * intersection) / (mask1.sum() + mask2.sum())
    return dice

# Hardcoded centroids as provided (X_bar = 268, Y_bar = 210, Z_bar = 0)
X_bar = 275
Y_bar = 200
Z_bar = 0

# Loop over degrees from -10 to 10
for DEG in range(9,15):
    ϕ = - DEG / 180 * np.pi  # Convert degree to radians for rotation

    # 3x3 Rotation matrix around the Z-axis
    RO = np.asarray([[np.cos(ϕ), -np.sin(ϕ), 0, 0],  
                     [np.sin(ϕ),  np.cos(ϕ), 0, 0],  
                     [     0,         0,     1, 0],  
                     [     0,         0,     0, 1]])

    # Constants for the centroids (using the hardcoded values)
    C1 = np.asarray([[1, 0, 0, X_bar],
                     [0, 1, 0, Y_bar],
                     [0, 0, 1, Z_bar],
                     [0, 0, 0,    1]])
    C2 = np.linalg.inv(C1)

    # Full affine matrix (translation + rotation)
    AFF = C1 @ RO @ C2 @ TR
    INV = np.linalg.inv(AFF)

    # Apply the inverse transformation to the wrong GT mask
    transformed_wrong_mask = affine_transform(wrong_mask.astype(np.float32), INV[:3, :3], offset=INV[:3, 3])
    transformed_wrong_mask = np.round(transformed_wrong_mask).astype(np.uint8)  # Ensure it's binary

    # Calculate Dice coefficient between the transformed wrong GT and the correct GT2
    dice_coefficient = calculate_dice(transformed_wrong_mask, good_mask)
    print(f"Dice Coefficient for degree {DEG} between transformed GT_old and GT2: {dice_coefficient:.4f}")


Original Computed Translation Vector: [66.58178962 38.56836791 14.99964341]
Affine matrix (with translation only):
[[ 1.          0.          0.         66.58178962]
 [ 0.          1.          0.         38.56836791]
 [ 0.          0.          1.         14.99964341]
 [ 0.          0.          0.          1.        ]]
Dice Coefficient for degree 9 between transformed GT_old and GT2: 0.9202
Dice Coefficient for degree 10 between transformed GT_old and GT2: 0.9183
Dice Coefficient for degree 11 between transformed GT_old and GT2: 0.9153
Dice Coefficient for degree 12 between transformed GT_old and GT2: 0.9114
Dice Coefficient for degree 13 between transformed GT_old and GT2: 0.9067
Dice Coefficient for degree 14 between transformed GT_old and GT2: 0.9016


With the centroids we found in 3d slicer

In [20]:
import nibabel as nib
import numpy as np
from scipy.ndimage import affine_transform

# Load the images
wrong_GT = nib.load('segthor_original/train/Patient_27/GT_old.nii.gz')
good_GT = nib.load('segthor_original/train/Patient_27/GT2.nii.gz')

wrong_GT_data = wrong_GT.get_fdata()
good_GT_data = good_GT.get_fdata()

# Extract only the heart segmentation (label 2)
wrong_mask = (wrong_GT_data == 2)
good_mask = (good_GT_data == 2)

# Get the coordinates of the heart voxels
coords_wrong = np.array(np.nonzero(wrong_mask)).T
coords_good = np.array(np.nonzero(good_mask)).T

# Ensure both arrays have the same number of points
min_points = min(len(coords_wrong), len(coords_good))
if len(coords_wrong) > min_points:
    coords_wrong = coords_wrong[np.random.choice(len(coords_wrong), min_points, replace=False)]
else:
    coords_good = coords_good[np.random.choice(len(coords_good), min_points, replace=False)]

# Compute the centroids (center of mass)
centroid_wrong = coords_wrong.mean(axis=0)
centroid_good = coords_good.mean(axis=0)

# Compute the translation vector and reverse it
translation_vector = centroid_good - centroid_wrong
print(f"Original Computed Translation Vector: {translation_vector}")

# Store the translation vector in a 4x4 affine transformation matrix (identity matrix with translation)
affine_matrix = np.eye(4)
affine_matrix[:3, 3] = translation_vector
print(f"Affine matrix (with translation only):\n{affine_matrix}")

TR = affine_matrix

# Calculate the Dice coefficient
def calculate_dice(mask1, mask2):
    """
    Calculate Dice coefficient between two binary masks.
    """
    intersection = np.logical_and(mask1, mask2).sum()
    dice = (2. * intersection) / (mask1.sum() + mask2.sum())
    return dice

# Hardcoded centroids as provided (X_bar = 268, Y_bar = 210, Z_bar = 0)
X_bar = -31
Y_bar = 22
Z_bar = 20

# Loop over degrees from -10 to 10
for DEG in range(-10, 10):
    ϕ = - DEG / 180 * np.pi  # Convert degree to radians for rotation

    # 3x3 Rotation matrix around the Z-axis
    RO = np.asarray([[np.cos(ϕ), -np.sin(ϕ), 0, 0],  
                     [np.sin(ϕ),  np.cos(ϕ), 0, 0],  
                     [     0,         0,     1, 0],  
                     [     0,         0,     0, 1]])

    # Constants for the centroids (using the hardcoded values)
    C1 = np.asarray([[1, 0, 0, X_bar],
                     [0, 1, 0, Y_bar],
                     [0, 0, 1, Z_bar],
                     [0, 0, 0,    1]])
    C2 = np.linalg.inv(C1)

    # Full affine matrix (translation + rotation)
    AFF = C1 @ RO @ C2 @ TR
    INV = np.linalg.inv(AFF)

    # Apply the inverse transformation to the wrong GT mask
    transformed_wrong_mask = affine_transform(wrong_mask.astype(np.float32), INV[:3, :3], offset=INV[:3, 3])
    transformed_wrong_mask = np.round(transformed_wrong_mask).astype(np.uint8)  # Ensure it's binary

    # Calculate Dice coefficient between the transformed wrong GT and the correct GT2
    dice_coefficient = calculate_dice(transformed_wrong_mask, good_mask)
    print(f"Dice Coefficient for degree {DEG} between transformed GT_old and GT2: {dice_coefficient:.4f}")


Original Computed Translation Vector: [66.5808327  38.56735908 14.99965244]
Affine matrix (with translation only):
[[ 1.          0.          0.         66.5808327 ]
 [ 0.          1.          0.         38.56735908]
 [ 0.          0.          1.         14.99965244]
 [ 0.          0.          0.          1.        ]]
Dice Coefficient for degree -10 between transformed GT_old and GT2: 0.2986
Dice Coefficient for degree -9 between transformed GT_old and GT2: 0.3564
Dice Coefficient for degree -8 between transformed GT_old and GT2: 0.4177
Dice Coefficient for degree -7 between transformed GT_old and GT2: 0.4815
Dice Coefficient for degree -6 between transformed GT_old and GT2: 0.5473
Dice Coefficient for degree -5 between transformed GT_old and GT2: 0.6149
Dice Coefficient for degree -4 between transformed GT_old and GT2: 0.6838
Dice Coefficient for degree -3 between transformed GT_old and GT2: 0.7534
Dice Coefficient for degree -2 between transformed GT_old and GT2: 0.8201
Dice Coeffici

KeyboardInterrupt: 

We obtained some reasonable results but were not really satisfied, in the end we used the affine matrix that was provided. As you can see in transform.py


### Finding the Affine Transformation

In this notebook, we explored using a rigid transformation followed by rotations around the Z-axis to align the wrongly transformed ground truth (GT_old) with the correct segmentation (GT2). 

1. **Rigid Registration**:
   - We used SimpleITK to perform rigid registration, where the fixed image was the correct segmentation (GT2) and the moving image was the wrongly transformed segmentation (GT_old).
   - The transformation matrix was saved as `rigid_transform.tfm`.

2. **Applying the Transformation**:
   - The saved transformation matrix was applied to the moving image, and the transformed image was saved as `GT_corrected_rigid.nii.gz`.

3. **Rotation and Dice Coefficient**:
   - We applied rotations around the Z-axis using the hardcoded centroids (`X_bar = 268, Y_bar = 210, Z_bar = 0`) and computed the Dice coefficient for each rotation.

In the end, we obtained some reasonable results, but we ultimately decided to use the affine matrix that was provided, as detailed in `transform.py`.
